In [25]:
from pyAudioAnalysis import audioFeatureExtraction as aFE, audioBasicIO, \
audioAnalysis,audioSegmentation,audioVisualization,audioTrainTest
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import numpy as np
import keras
from keras import models,layers
import os
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, GRU
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

def amplitude_in_time(wave_file,threshold_seconds,sampling_rate_Hz):
    
    """this function gives you amplitude in time with a threshold settings
    with PyAudioAnalysis data loading. """
    
    Fs, X = audioBasicIO.readAudioFile(wave_file)
    sampling_indexes = []
    data_new=[]
    np.arange(0,threshold_seconds*Fs,50)
    sampling_indexes=sampling_indexes[::-1]
    if len(X)/Fs > threshold_seconds:
        X = X[0:threshold_seconds*Fs]
        data_new=[]
        for i in sampling_indexes:
            data_new.append(X[i])
        data_new = data_new[::-1]
        data.append(data_new)

paths={'peterbertley_Audio/Atraining_murmur':0,
       'peterbertley_Audio/Atraining_normal':1,
       'peterbertley_Audio/Atraining_extrahls':2}

def create_amplitude_dataset(path_dict):
    """ To create a dataset of amplitudes of the same length"""
    for k,i in enumerate(paths.keys()):
        for audio_file in os.listdir(i):
            if audio_file.endswith(".wav"):
                amplitude_in_time(i+'/'+str(audio_file),5,50)
                target.append(list(paths.values())[k]) 

data=[]
target=[]
create_amplitude_dataset(paths)

# Data --> Data --> with amplitude (vector for sample)

In [20]:
def amplitude_data():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                #X_temp = X[1:]
                #data.append(X_temp-X[:-1])
                data.append(data)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        #data[k]=i[sampling_indexes]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

# Data --> Feature Extraction with FeatureExtraction of pyAudioAnalysis

In [ ]:
def feature_extraction_pyaudio():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
                data.append(F)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
                data.append(F)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                F, f_names = aFE.stFeatureExtraction(X, Fs, 0.050*Fs, 0.025*Fs)
                data.append(F)
                target.append(['2','extrahls'])
    #Data Preprocessing
    #Defining the threshold
    print('samples are ',len(data))
    list_lengths =[]
    for i in data:
        list_lengths.append(i.shape[1])
    threshold  =int(np.max(list_lengths)*.75)
    print('The threshold is',threshold,'and represent the number of measurement for the every single feature', 
       'of the 32 features for each sample')
    return X_train,X_test,y_train,y_test

# Data --> Feature Extraction with stft of librosa - Windowing

In [2]:
def stft_librosa_data():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                y, sr = librosa.load('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(y)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                y, sr = librosa.load('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(y)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                y, sr = librosa.load('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(y)
                target.append(['2','extrahls'])
    list=[]
    for k,i in enumerate(data):
        list.append(len(data[k]))
    threshold = 160000  
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))
    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
    data = np.array(data)
    data_new = []
    for k,i in enumerate(data):
        data_new.append(librosa.core.stft(i,n_fft=2048,win_length=2048))
    data = np.array(data_new)
    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

    print('number of sample is',len(X_train),'sample shape is',X_train[0].shape)
    return X_train,X_test,y_train,y_test

# ANN - Feed Forward - Data with Windowing

In [ ]:
#X_train,X_test,y_train,y_test = stft_librosa_data()
X_train,X_test,y_train,y_test = amplitude_data()
#X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
X_train.shape

In [ ]:
'''len(X_train[0])
indexes_filtering = np.arange(0,4410,5)
indexes_filtering
print(X_train.shape)
X_train = X_train[:,indexes_filtering]
print(X_train.shape)
print(X_test.shape)
X_test = X_test[:,indexes_filtering]
print(X_test.shape)'''

In [ ]:
model = models.Sequential()
model.add(layers.Dense(4000, activation='relu',input_shape=(4410,)))#X_train.shape[1],X_train.shape[2],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2000, activation='relu'))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(300, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
#model.add(layers.Flatten())
model.add(layers.Dense(3,activation='softmax'))
model.summary()

In [ ]:
from keras import losses
from keras import optimizers

In [ ]:
sgd = optimizers.rmsprop(lr=00.1)#, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
#model.compile(optimizer='adam',loss=losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=20,
    batch_size=64)

In [ ]:
model.evaluate(X_test,y_test)

# Stacked LSTM for STFT windows

In [ ]:
X_train,X_test,y_train,y_test = stft_librosa_data()
#X_train,X_test,y_train,y_test = amplitude_data()
#X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(1000,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(500,activation='relu',return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,epochs=10)   #,batch_size=3

In [ ]:
model.evaluate(X_test,y_test)

# ANN - Feed Forward - Data with amplitude

In [ ]:
X_train,X_test,y_train,y_test = stft_librosa_data()
#X_train,X_test,y_train,y_test = amplitude_data()
#X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
model = models.Sequential()
model.add(layers.Dense(2000, activation='relu',input_shape=(4410)))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=20,
    batch_size=8)

In [ ]:
model.evaluate(X_test,y_test)

# ANN - Feed Forward - Data with Feature Extraction¶

In [ ]:
X_train,X_test,y_train,y_test = stft_librosa_data()
#X_train,X_test,y_train,y_test = amplitude_data()
#X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
model = models.Sequential()
model.add(layers.Dense(20000, activation='relu',input_shape=(34,269,)))
model.add(layers.Dense(5000, activation='relu'))
model.add(layers.Dense(2000, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(3,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
X_train.shape

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=10,
    batch_size=20)

# Stacked LSTM for Feature Extraction

In [ ]:
#print(X_train.shape,y_train.shape)
#X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])

In [ ]:
X_train,X_test,y_train,y_test = stft_librosa_data()
#X_train,X_test,y_train,y_test = amplitude_data()
#X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(1000,input_shape=(34,269,),return_sequences=True))
model.add(LSTM(500,activation='relu',return_sequences=False))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,epochs=10)   #,batch_size=3

In [ ]:
print(X_test.shape)
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
model.evaluate(X_test,y_test)

# Neural Network with amplitude

In [ ]:
#X_train,X_test,y_train,y_test = stft_librosa_data()
#X_train,X_test,y_train,y_test = amplitude_data()
X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
X_train.shape

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()

model.add(LSTM(500,input_shape=(34,240),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(300,activation='relu',return_sequences=True))    
model.add(LSTM(100,activation='relu',return_sequences=False))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,batch_size=4,epochs=30)   #,batch_size=3

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.save('lstm1.h5')   #del model  (deletes the existing model)  #model = load_model('my_model.h5')
# returns a compiled model
# identical to the previous one

In [ ]:
y,sr = librosa.load('my_heart_3.wav')

In [ ]:
print(len(y),sr)

In [ ]:
stft = librosa.core.stft(y,n_fft=2048,win_length=2048)

In [ ]:
print(type(stft))
stft.shape

In [ ]:
np.abs(stft).shape

In [ ]:
np.angle(stft).shape

# RNN - GRU - Data with Windowing

In [ ]:
X_train,X_test,y_train,y_test = stft_librosa_data()
#X_train,X_test,y_train,y_test = amplitude_data()
#X_train,X_test,y_train,y_test = feature_extraction_pyaudio()

In [ ]:
model = Sequential()

model.add(GRU(500,input_shape=(1025, 313),return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(300,activation='relu',return_sequences=True)) 
model.add(Dropout(0.2))
model.add(GRU(100,activation='relu',return_sequences=False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=10,
    batch_size=100)

In [ ]:
model.evaluate(X_test,y_test)

# Random Tree Classifier

In [ ]:
target=[]
i=0
data=[]

for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
            data.append(X)
            target.append('0')

for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
            data.append(X)
            target.append('1')
threshold = 4*Fs   #5 seconds threshols
sampling_indexes = np.arange(0,threshold,50)
drop=[]
for k,i in enumerate(data):
    if len(i)<threshold:
        drop.append(k)
drop = drop[::-1]  #reversing the order for the dropping
print('shape before dropping:',len(data),len(target))
for i in drop:
    data.pop(i)
    target.pop(i)
print('shape after dropping:',len(data),len(target))

In [ ]:
for k,i in enumerate(data):
    data[k]=i[:threshold]
    data[k]=i[sampling_indexes]
data = np.array(data)

In [ ]:
#target = to_categorical(target)
print('final categorical target shape is:')

# Splitting data in training and test set

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

In [ ]:
model_ExTC = ExtraTreesClassifier(n_estimators=1000,criterion='entropy',n_jobs=-1,random_state=2)
model_ExTC.fit(X_train,y_train)
print(model_ExTC.score(X_test,y_test))
confusion_matrix(model_ExTC.predict(X_test),y_test)

In [ ]:
model_RFC = RandomForestClassifier(n_estimators=1000,criterion='entropy',n_jobs=-1,random_state=2)
model_RFC.fit(X_train,y_train)
print(model_RFC.score(X_test,y_test))
confusion_matrix(model_RFC.predict(X_test),y_test)

In [ ]:
model_DTC = DecisionTreeClassifier(criterion='entropy',random_state=2)
model_DTC.fit(X_train,y_train)
print(model_DTC.score(X_test,y_test))
confusion_matrix(model_DTC.predict(X_test),y_test)

In [ ]:
model_GBC = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                       n_estimators=100, subsample=1.0, criterion='friedman_mse')
model_GBC.fit(X_train,y_train)
model_GBC.score(X_test,y_test)
confusion_matrix(model_GBC.predict(X_test),y_test)

In [ ]:
model_GNB = GaussianNB()
model.fit(X_train,y_train)
model.score(X_test,y_test)
confusion_matrix(model.predict(X_test),y_test)

In [36]:
target=[]
i=0
data=[]

for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
            data.append(X)
            target.append('0')
            
for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
            data.append(X)
            target.append(['1'])

threshold = 5*Fs   #5 seconds threshols
drop=[]
for k,i in enumerate(data):
    if len(i)<threshold:
        drop.append(k)
drop = drop[::-1]  #reversing the order for the dropping
print('shape before dropping:',len(data),len(target))
for i in drop:
    data.pop(i)
    target.pop(i)
print('shape after dropping:',len(data),len(target))

for k,i in enumerate(data):
    data[k]=i[:threshold]
    #data[k]=i[sampling_indexes]
data = np.array(data)

# StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)

target = to_categorical(target)

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)

shape before dropping: 53 53
shape after dropping: 47 47


/Users/luigibungaro/dsr/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


ValueError: setting an array element with a sequence.

In [37]:
def classification_models():
    
    model_ExTC = ExtraTreesClassifier(n_estimators=1000,criterion='entropy',n_jobs=-1,random_state=2)
    model_ExTC.fit(X_train,y_train)
    print('model_ExTC',model_ExTC.score(X_test,y_test))
    print(confusion_matrix(model_ExTC.predict(X_test),y_test))
    
    model_RFC = RandomForestClassifier(n_estimators=1000,criterion='entropy',n_jobs=-1,random_state=2)
    model_RFC.fit(X_train,y_train)
    print('model_RFC',model_RFC.score(X_test,y_test))
    print(confusion_matrix(model_RFC.predict(X_test),y_test))
          
    model_DTC = DecisionTreeClassifier(criterion='entropy',random_state=2)
    model_DTC.fit(X_train,y_train)
    print('model_DTC',model_DTC.score(X_test,y_test))
    print(confusion_matrix(model_DTC.predict(X_test),y_test))
          
    model_GBC = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                       n_estimators=100, subsample=1.0, criterion='friedman_mse')
    model_GBC.fit(X_train,y_train)
    print('model_GBC',model_GBC.score(X_test,y_test))
    print(confusion_matrix(model_GBC.predict(X_test),y_test))
          
    model_GNB = GaussianNB()
    model_GNB.fit(X_train,y_train)
    print('model_GNB',model_GNB.score(X_test,y_test))
    print(confusion_matrix(model_GNB.predict(X_test),y_test))

In [38]:
classification_models()

ValueError: setting an array element with a sequence

In [6]:
X_train,X_test,y_train,y_test = amplitude_data()

model_RFC = RandomForestClassifier(n_estimators=10000,criterion='entropy',n_jobs=-1,random_state=2)
model_RFC.fit(X_train,y_train)
print('model_RFC',model_RFC.score(X_test,y_test))
#print(confusion_matrix(model_RFC.predict(X_test),y_test))

shape before dropping: 84 84
shape after dropping: 76 76


/Users/luigibungaro/dsr/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


final shape of your data is:  (76, 220500)
final categorical target shape is: (76, 3)


KeyboardInterrupt: 

In [9]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()

model.add(LSTM(3000,input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(1000,activation='relu',return_sequences=True))    
model.add(LSTM(200,activation='relu',return_sequences=False))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,batch_size=234,epochs=20)

Epoch 1/20


In [ ]:
model = models.Sequential()
model.add(layers.Dense(300, activation='relu',input_shape=(X_train.shape[1],1)))#X_train.shape[1],X_train.shape[2],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(3,activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(X_train, y_train,batch_size=1056,epochs=10) 

In [3]:
model.evaluate(X_test,y_test)

NameError: name 'model' is not defined

In [21]:
X_train,X_test,y_train,y_test = amplitude_data()

shape before dropping: 84 84
shape after dropping: 76 76


/Users/luigibungaro/dsr/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


final shape of your data is:  (76, 220500)
final categorical target shape is: (76, 3)


In [22]:
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [23]:
y_train = y_train.reshape(y_train.shape[0],1,y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0],1,y_test.shape[1])

In [24]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()

model.add(LSTM(32,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
#model.add(Dropout(0.5))
model.add(LSTM(8,activation='relu',return_sequences=True))    
#model.add(LSTM(100,activation='relu',return_sequences=False))
#model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,batch_size=132,epochs=30)   #,batch_size=3

model.evaluate(X_test,y_test)

Epoch 1/30
45/45 [==============================] - 4s 94ms/step - loss: 1.0939 - acc: 0.3778
Epoch 2/30
45/45 [==============================] - 1s 19ms/step - loss: 0.9847 - acc: 0.6444
Epoch 3/30
45/45 [==============================] - 1s 16ms/step - loss: 0.9418 - acc: 0.7556
Epoch 4/30
45/45 [==============================] - 1s 16ms/step - loss: 0.9015 - acc: 0.8222
Epoch 5/30
45/45 [==============================] - 1s 18ms/step - loss: 0.8737 - acc: 0.8889
Epoch 6/30
45/45 [==============================] - 1s 17ms/step - loss: 0.8601 - acc: 0.8889
Epoch 7/30
45/45 [==============================] - 1s 17ms/step - loss: 0.8489 - acc: 0.8667
Epoch 8/30
45/45 [==============================] - 1s 16ms/step - loss: 0.8415 - acc: 0.8667
Epoch 9/30
45/45 [==============================] - 1s 17ms/step - loss: 0.8347 - acc: 0.8667
Epoch 10/30
45/45 [==============================] - 1s 16ms/step - loss: 0.8286 - acc: 0.8667
Epoch 11/30
45/45 [==============================] - 1s 17m

[1.0271623134613037, 0.5161290168762207]

In [ ]:
for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
     if item.endswith(".wav"):
            Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
            data.append(X)
            target.append('1')